# Dependency visualizations
Explore dependency parsed sentences with spacy

In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [116]:
sent = nlp("the phone has good battery life")
for chunk in sent.noun_chunks:
    print(chunk)
displacy.render(sent, style='dep', jupyter = True)

the phone
good battery life


In [123]:
sent = nlp("The paper is cheap, weak, and flimsy")
displacy.render(sent, style='dep', jupyter = True)

In [97]:
sent = nlp("Black matte finish has good construction design and is cool.")
displacy.render(sent, style='dep', jupyter = True)

In [114]:
sent = nlp("Marketer brought diversity action alleging that manufacturer committed material breach of contract, deliberately and intentionally breached its duty of good faith and fair dealing, and breached its promise to provide appropriate advertising and promotion for sale of its products in IsraelCLAIM.")
displacy.render(sent, style='dep', jupyter = True)

In [233]:
sent = nlp("It is a great tutu and at a really great price.")
for word in sent:
    print(word, word.pos_)
displacy.render(sent, style='dep', jupyter = True)

It PRON
is VERB
a DET
great ADJ
tutu NOUN
and CCONJ
at ADP
a DET
really ADV
great ADJ
price NOUN
. PUNCT


### testing extraction code

In [243]:
link_verbs = ['is', '\'s', 'am', 'are', 'wa', 'were', 'look', 'sound', 'smell', 'tast','feel','appear',
              'remain','seem','ha','had', 'have']

# personal pronouns - pretty much all pronouns except "it", "this", and "that"
pronouns = ["I", "you", "he", "she"," they", "me", "you", "him", "her", "my", "mine", "your", "yours",
            "his", "her", "hers", "who", "whom", "whose", "what", "which",  "another", "each", 
            "everything", "nobody", "either", "someone", "who", "whom", "whose", "which", 
           "myself", "yourself", "himself", "herself", "itself"]

from nltk.stem import PorterStemmer
stem = PorterStemmer()


### I need to check the linking verbs in their root form.  I.e., look vs. looks vs. looked 

In [253]:
import string
import re
sent = nlp("The paper is flimsy, weak, and cheap.")

# extracts the children for a verb (removes punctuation and verifies its not part of original chunk)
def extractChildren(verb, chunk):
    return [child for child in verb.children 
            if child.text not in string.punctuation 
            and child.text not in [word.text for word in chunk]]


# identify individual aspects
# returns all potential adjectives excluding proper and pronouns
def extractIndividualAspects(noun_chunk, feature, indiv_aspects):
   
    if any([word for word in noun_chunk if word.pos_ in ['PRON', 'PROPN']]):
        indiv_aspects.append(feature)

    else:
        indiv_aspects += [noun_chunk.text, feature]
        
    return indiv_aspects


# noun chunk links to adjectives
def link2adj(child, aspects, indiv_aspects):
    # if adjective then we know its part of a feature
    if child.pos_ == "ADJ":
        
        # how can I add logic that captures "to verb" in an easy way
        child_text = None
        for c in child.children:
            if c.pos_ == "VERB" and list(c.children)[0].text == "to":
                child_text = ' '.join([word for word in [child.text, "to", c.text]])
        
        if child_text == None:
            child_text = child.text
        
        aspects.append(' '.join([word for word in [chunk.text, verb.text, child_text]]))
        
        # get the individual aspects too
        indiv_aspects = extractIndividualAspects(chunk, child_text, indiv_aspects)


    return aspects, indiv_aspects

# noun chunk links to noun then adjective
def link2noun(child, aspects, indiv_aspects, skips, sent):
    if child.pos_ == "NOUN":
        # collect the entire chunk 
        feat_chunk = [chunk.text for chunk in list(sent.noun_chunks)
                      if re.search(child.text, chunk.text)][0]
        
        # add to aspects and skips
        aspects.append(' '.join([word for word in [chunk.text, verb.text, feat_chunk]]))
        skips.append(feat_chunk)
        
        # get the individual aspects too
        indiv_aspects = extractIndividualAspects(chunk, feat_chunk, indiv_aspects)

        
    return aspects, indiv_aspects, skips

# recursive algorithm to extract conjugations 
def extractCONJ(child, aspects, indiv_aspects):

    # see if we need to go further down the tree
    # if there are further children (multiple conjugations then we continue traversing to the bottom)
    child_children = extractChildren(child, chunk)
    if len(child_children) > 0:
        for c in child_children:
            aspects, indiv_aspects = extractCONJ(c, aspects, indiv_aspects)

    # otherwise extract the aspects
    # once at the bottom, we check to see if our criteria are met then add aspects
    # then we will come back up the tree
    if child.dep_ in ['acomp',  'conj', "xcomp", "attr"]:
        aspects, indiv_aspects = link2adj(child, aspects, indiv_aspects)
        aspects, indiv_aspects, _ = link2noun(child, aspects, indiv_aspects, skips, sent)
            
    return aspects, indiv_aspects

        
aspects = [] # aspect clauses
indiv_aspects = [] # individual aspects
skips = [] # skips already reviewed noun chunks 
for chunk in sent.noun_chunks:
    
    # skip if noun chunk was already reviewed or if the noun chunk is a pronoun
    if chunk.text in skips or any([word for word in chunk if word.text.lower() in pronouns]):
        continue
    if (chunk.root.dep_ == "nsubj") and (stem.stem(chunk.root.head.text) in link_verbs):
        verb = chunk.root.head
        # get children for each verb, removing core noun chunk and any punctuation
        children = extractChildren(verb, chunk)

        for child in children:
            if child.dep_ in ['acomp', 'dobj', "xcomp", "attr"]: # unsure about including dobj, but can leave for now
                # create aspect if links to adjective
                aspects, indiv_aspects = link2adj(child, aspects, indiv_aspects)
                
                # create aspect if links to other noun chunk
                aspects, indiv_aspects, skips = link2noun(child, aspects, indiv_aspects, skips, sent)
                    
                # if criteria are met, we check to see if there are any conjugations
                # recursively call functions to extract conjugations
                # since it is conjugations, we do not need to worry about skips, since it will have been captured already
                child_children = extractChildren(child, chunk)
                for c in child_children:
                    aspects, indiv_aspects = extractCONJ(c, aspects, indiv_aspects)

print(aspects, indiv_aspects)

['The paper is flimsy', 'The paper is cheap', 'The paper is weak'] ['The paper', 'flimsy', 'The paper', 'cheap', 'The paper', 'weak']


In [242]:
for word in sent:
    print(word, word.dep_)

It nsubj
's ROOT
really advmod
great acomp


In [211]:
chunk

the phone